In [1]:
import pandas as pd

In [2]:
meta_dataset = pd.read_csv("./performance_meta_dataset.csv", index_col=0)
metafeats_fn = "../RecSys2019_DeepLearning_Evaluation/Metafeatures/Metafeatures.csv"
metafeats = pd.read_csv(metafeats_fn)
join_cols = ["dataset_name", "split_name"]
metafeats.columns = ["f__{}".format(col) if col not in join_cols else col for col in metafeats.columns]
del metafeats["split_name"]
metafeats = meta_dataset.merge(metafeats, on="dataset_name", how='left')
metafeats

,alg_name,dataset_name,max_test_metric_ARHR_ALL_HITS_cut_1,max_test_metric_ARHR_ALL_HITS_cut_10,max_test_metric_ARHR_ALL_HITS_cut_15,max_test_metric_ARHR_ALL_HITS_cut_2,max_test_metric_ARHR_ALL_HITS_cut_20,max_test_metric_ARHR_ALL_HITS_cut_3,max_test_metric_ARHR_ALL_HITS_cut_30,max_test_metric_ARHR_ALL_HITS_cut_4,...,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_mean,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_median,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_min,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_mode,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_skewness,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_std,f__num_interactions,f__num_items,f__num_users,f__sparsity
0,CoClustering,AnimeReader,0.000043,4.104161e-04,0.000673,0.000101,0.000898,0.000134,0.001251,0.000174,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
1,CoClustering,BookCrossingReader,0.000000,2.363228e-05,0.000024,0.000000,0.000024,0.000015,0.000025,0.000015,...,109.713976,29.0,1.0,8.0,68.610767,629.711043,323443,340556,22568,0.999958
2,CoClustering,CiaoDVDReader,0.000211,3.162555e-04,0.000316,0.000316,0.000327,0.000316,0.000336,0.000316,...,39.321526,13.0,1.0,5.0,13.294764,133.168783,47102,16121,4743,0.999384
3,CoClustering,DatingReader,0.000000,8.208624e-07,0.000001,0.000000,0.000001,0.000000,0.000001,0.000000,...,749.554740,403.0,24.0,114.0,23.291594,2137.414663,17088628,168791,135359,0.999252
4,CoClustering,EpinionsReader,0.000000,8.775933e-06,0.000009,0.000000,0.000009,0.000000,0.000010,0.000009,...,82.886861,36.0,1.0,5.0,8.135884,167.403345,592236,139738,28487,0.999851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,UserKNNCF_tversky,Movielens20MReader,0.001610,8.872977e-03,0.011534,0.002762,0.013791,0.003720,0.016900,0.004657,...,501.883546,241.5,9.0,77.0,5.670739,767.111703,19723277,27278,138493,0.994779
492,UserKNNCF_tversky,MovielensHetrec2011Reader,0.002840,1.003369e-02,0.011235,0.004496,0.012326,0.005679,0.013881,0.006981,...,1384.954567,956.0,9.0,163.0,2.031122,1401.603148,851372,10109,2113,0.960142
493,UserKNNCF_tversky,NetflixPrizeReader,0.001015,2.703064e-03,0.003164,0.001395,0.003579,0.001637,0.004170,0.001850,...,752.338781,352.0,1.0,65.0,4.457685,1048.358240,99521398,17770,476694,0.988251
494,UserKNNCF_tversky,RecipesReader,0.000085,3.495172e-03,0.003918,0.000987,0.004182,0.001697,0.004487,0.002190,...,108.350440,19.0,1.0,5.0,22.995823,584.417930,819642,231637,35464,0.999900


In [3]:
list(meta_dataset.columns)

['alg_name',
 'dataset_name',
 'max_test_metric_ARHR_ALL_HITS_cut_1',
 'max_test_metric_ARHR_ALL_HITS_cut_10',
 'max_test_metric_ARHR_ALL_HITS_cut_15',
 'max_test_metric_ARHR_ALL_HITS_cut_2',
 'max_test_metric_ARHR_ALL_HITS_cut_20',
 'max_test_metric_ARHR_ALL_HITS_cut_3',
 'max_test_metric_ARHR_ALL_HITS_cut_30',
 'max_test_metric_ARHR_ALL_HITS_cut_4',
 'max_test_metric_ARHR_ALL_HITS_cut_40',
 'max_test_metric_ARHR_ALL_HITS_cut_5',
 'max_test_metric_ARHR_ALL_HITS_cut_50',
 'max_test_metric_ARHR_ALL_HITS_cut_6',
 'max_test_metric_ARHR_ALL_HITS_cut_7',
 'max_test_metric_ARHR_ALL_HITS_cut_8',
 'max_test_metric_ARHR_ALL_HITS_cut_9',
 'max_test_metric_AVERAGE_POPULARITY_cut_1',
 'max_test_metric_AVERAGE_POPULARITY_cut_10',
 'max_test_metric_AVERAGE_POPULARITY_cut_15',
 'max_test_metric_AVERAGE_POPULARITY_cut_2',
 'max_test_metric_AVERAGE_POPULARITY_cut_20',
 'max_test_metric_AVERAGE_POPULARITY_cut_3',
 'max_test_metric_AVERAGE_POPULARITY_cut_30',
 'max_test_metric_AVERAGE_POPULARITY_cut_4',


In [4]:
# Algorithm selection

def rank_algorithms(test_datasets, metric_name):
    """Compute algorithm ranks for each dataset"""
    # Sanity check to prevent leakage
    for test_dataset in test_datasets:
        assert test_dataset in meta_dataset['dataset_name'].values
    filtered_dataset = meta_dataset[~meta_dataset['dataset_name'].isin(test_datasets)]
    
    all_ranks = []
    for dataset_name, dataset_performance in filtered_dataset.groupby("dataset_name"):
        dataset_performance["rank"] = dataset_performance["max_test_metric_" + metric_name].rank(method='min', ascending=False)
        dataset_performance.set_index("alg_name", inplace=True)
        dataset_performance = dataset_performance[["rank"]]
        dataset_performance = dataset_performance.rename(columns={"rank": dataset_name})
        all_ranks.append(dataset_performance)
        
    ranked_algs = pd.concat(all_ranks, axis=1)
    return ranked_algs

def select_algs(test_datasets, metric_name, num_algs=10):
    """Select num_algs algorithm with best mean rank"""
    return list(ranked_algs.T.mean().sort_values().iloc[:num_algs].index)

In [5]:
# Metafeature selection

def compute_feature_corrs(test_datasets, metric_name, selected_algs):
    """Compute correlation between each metafeature and the desired metric for all selected algorithms.
    Dataframe result is num_features x num_algorithms."""
    all_features = [col for col in metafeats.columns if col.startswith("f__")]
    # Sanity check to prevent leakage
    for test_dataset in test_datasets:
        assert test_dataset in metafeats['dataset_name'].values
    filtered_metafeats = metafeats[~metafeats['dataset_name'].isin(test_datasets)]
    
    all_cors = []
    for alg in selected_algs:
        filtered_results = filtered_metafeats.loc[(filtered_metafeats["alg_name"] == alg)]
        alg_cors = filtered_results[all_features].corrwith(filtered_results["max_test_metric_" + metric_name],
                                                          method="spearman")
        alg_cors.name = alg
        all_cors.append(alg_cors)
    all_cors = pd.concat(all_cors, axis=1).abs()
    return all_cors

def select_features(test_datasets, metric_name, selected_algs, num_feats=10):
    """Select num_feats features. Greedy scheme. At each step, we compute the best correlations
    across all metafeatures for each algorithm so far. We add whichever metafeature can obtain the maximum
    improvement across any single one of the best correlations for the selected algorithms."""
    all_cors = compute_feature_corrs(test_datasets, metric_name, selected_algs)
    
    selected_feats = [all_cors.max(axis=1).idxmax()]
    
    while len(selected_feats) < num_feats:
        # Current best correlations
        current_best_cors = all_cors.loc[selected_feats].max(axis=0)
        # Pick whichever feature results in the highest maximum improvement on the current best correlations
        selected_feats.append((
            all_cors.loc[~all_cors.index.isin(selected_feats)] - current_best_cors)
            .max(axis=1)
            .idxmax())
    return selected_feats

In [6]:
# Assume some split
test_datasets = ["AnimeReader", "CiaoDVDReader"]
# Assume some metric
metric_name = "ARHR_ALL_HITS_cut_1"

In [7]:
ranked_algs = rank_algorithms(test_datasets, metric_name)
ranked_algs

,BookCrossingReader,DatingReader,EpinionsReader,FilmTrustReader,FrappeReader,GowallaReader,Jester2Reader,LastFMReader,MarketBiasAmazonReader,MarketBiasModClothReader,MovieTweetingsReader,Movielens100KReader,Movielens10MReader,Movielens1MReader,Movielens20MReader,MovielensHetrec2011Reader,NetflixPrizeReader,RecipesReader,WikilensReader
alg_name,,,,,,,,,,,,,,,,,,,
CoClustering,21.0,20.0,23.0,24.0,24.0,14.0,20.0,23.0,22.0,23.0,21.0,23.0,22.0,25.0,21.0,23.0,NaN,20.0,25.0
GlobalEffects,19.0,17.0,19.0,25.0,24.0,14.0,21.0,23.0,18.0,21.0,17.0,23.0,17.0,23.0,16.0,21.0,17.0,18.0,22.0
IALSRecommender,9.0,9.0,9.0,16.0,20.0,NaN,24.0,11.0,14.0,19.0,11.0,12.0,12.0,12.0,11.0,16.0,NaN,10.0,11.0
ItemKNNCF_asymmetric,5.0,7.0,6.0,8.0,1.0,1.0,6.0,13.0,10.0,7.0,3.0,5.0,6.0,5.0,5.0,8.0,4.0,6.0,2.0
ItemKNNCF_cosine,1.0,1.0,4.0,5.0,2.0,5.0,1.0,3.0,3.0,3.0,4.0,7.0,5.0,4.0,1.0,3.0,3.0,1.0,6.0
ItemKNNCF_dice,2.0,3.0,1.0,2.0,5.0,3.0,10.0,1.0,7.0,12.0,7.0,1.0,2.0,3.0,2.0,2.0,1.0,2.0,1.0
ItemKNNCF_euclidean,6.0,8.0,10.0,1.0,11.0,NaN,11.0,8.0,13.0,16.0,10.0,3.0,8.0,10.0,9.0,7.0,8.0,13.0,2.0
ItemKNNCF_jaccard,3.0,4.0,3.0,4.0,5.0,4.0,8.0,5.0,7.0,12.0,9.0,2.0,7.0,1.0,3.0,5.0,2.0,2.0,2.0
ItemKNNCF_tversky,3.0,2.0,2.0,3.0,2.0,2.0,8.0,2.0,6.0,7.0,5.0,4.0,4.0,2.0,4.0,1.0,5.0,5.0,2.0


In [8]:
selected_algs = select_algs(test_datasets, metric_name)
selected_algs

['ItemKNNCF_cosine',
 'ItemKNNCF_dice',
 'ItemKNNCF_tversky',
 'ItemKNNCF_jaccard',
 'ItemKNNCF_asymmetric',
 'SLIM_BPR_Cython',
 'SLIMElasticNetRecommender',
 'P3alphaRecommender',
 'EASE_R_Recommender',
 'ItemKNNCF_euclidean']

In [9]:
feature_corrs = compute_feature_corrs(test_datasets, metric_name, selected_algs)
feature_corrs

,ItemKNNCF_cosine,ItemKNNCF_dice,ItemKNNCF_tversky,ItemKNNCF_jaccard,ItemKNNCF_asymmetric,SLIM_BPR_Cython,SLIMElasticNetRecommender,P3alphaRecommender,EASE_R_Recommender,ItemKNNCF_euclidean
f__dist_feature__kind_item__pre_agg_func_count__agg_func_Gini,0.145614,0.050877,0.054386,0.064912,0.100000,0.236326,0.050000,0.062951,0.063636,0.023736
f__dist_feature__kind_item__pre_agg_func_count__agg_func_kurtosis,0.471930,0.519298,0.524561,0.533333,0.580702,0.442724,0.271429,0.316821,0.100000,0.465428
f__dist_feature__kind_item__pre_agg_func_count__agg_func_max,0.050877,0.138596,0.143860,0.098246,0.024561,0.025800,0.189286,0.287926,0.000000,0.050568
f__dist_feature__kind_item__pre_agg_func_count__agg_func_mean,0.354386,0.324561,0.321053,0.354386,0.401754,0.430341,0.007143,0.038184,0.009091,0.304438
f__dist_feature__kind_item__pre_agg_func_count__agg_func_median,0.300048,0.294690,0.295583,0.333982,0.413459,0.360266,0.063296,0.024048,0.050578,0.305306
...,...,...,...,...,...,...,...,...,...,...
f__dist_feature__kind_user__pre_agg_func_sum__agg_func_std,0.147368,0.150877,0.166667,0.159649,0.314035,0.215686,0.525000,0.273478,0.554545,0.166151
f__num_interactions,0.280702,0.349123,0.354386,0.310526,0.245614,0.236326,0.332143,0.510836,0.300000,0.207430
f__num_items,0.600000,0.663158,0.663158,0.654386,0.671930,0.640867,0.503571,0.543860,0.490909,0.562436
f__num_users,0.317544,0.438596,0.449123,0.410526,0.303509,0.244582,0.303571,0.486068,0.181818,0.300310


In [10]:
selected_feats = select_features(test_datasets, metric_name, selected_algs)
selected_feats

['f__dist_feature__kind_item__pre_agg_func_mean__agg_func_max',
 'f__num_items',
 'f__dist_feature__kind_user__pre_agg_func_mean__agg_func_mean',
 'f__dist_feature__kind_rating__pre_agg_func_None__agg_func_mean',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_mean',
 'f__dist_feature__kind_user__pre_agg_func_count__agg_func_kurtosis',
 'f__dist_feature__kind_user__pre_agg_func_mean__agg_func_median',
 'f__dist_feature__kind_rating__pre_agg_func_None__agg_func_median',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_median',
 'f__num_interactions']

In [11]:
ranked_algs.T.mean().sort_values().iloc[:10]

alg_name
ItemKNNCF_cosine             3.263158
ItemKNNCF_dice               3.526316
ItemKNNCF_tversky            3.631579
ItemKNNCF_jaccard            4.631579
ItemKNNCF_asymmetric         5.684211
SLIM_BPR_Cython              5.944444
SLIMElasticNetRecommender    6.933333
P3alphaRecommender           7.611111
EASE_R_Recommender           8.272727
ItemKNNCF_euclidean          8.555556
dtype: float64